# GLOBAL SUPERSTORE DATASET SALES ANALYSIS
By Raju Vaneshwar Nareshwar

## Table of contents
1. [Task 1](#task1)

    1.1 [Load first 10 records](#load-first-10-records)
    
    1.2 [Understanding the dataset](#understanding-dataset)

2. [Assess the data](#assess)

    2.1 [Meta Data](#metadata)
    
    2.2 [Assessment Summary](#summary)

3. [Data Cleaning](#clean)

4. [Analysis and Data Visualization](#analysis)

    4.1 [Product Analysis](#product)

    4.2 [Segment Analysis](#segment)

    4.3 [Geographical market location Analysis](#market)

    4.4 [Shipping](#shipping)

    4.5 [Time Series Analysis](#time)

5. [Insights](#insights)

    5.1 [Findings](#findings)

    5.2 [Limitations](#limitation)

    5.3 [Recommendations](#recommendation)

## 1. Task 1  <a id='task1'></a>

### 1.1 Load first 10 records <a id='load-first-10-records'></a>

In [ ]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import ticker as mtick
import seaborn as sns

# Letting pandas to show max columns
pd.set_option('display.max_columns', None)

In [ ]:
# Reading CSV file and assigning into a dataframe ss_data
global_super_store_data = pd.read_csv('sample-superstore-2023-T3.csv')

# Set the head to 10 to retrieve the first 10 records
first_10_rows = global_super_store_data.head(n=10)
first_10_rows

### 1.2 Understanding of the dataset <a id='understanding-dataset'></a>

Using info() and describe() function to get the descriptive statistics

In [ ]:
# Get the metadata information about the dataset
global_super_store_data.info()

# Get descriptive statistics on the dataset
global_super_store_data.describe()

The primary key of these records are a system-generated, and denoted as column: *RowID*

The datatypes of the dataset are following:
* int64(1)
* float64(2)
* object(18)

A few records of *Quantity* and *Profit* columns has the datatype of object, but it must be float64, thus needs to be cleansed or transformed.  
*Ship Date* and *Order Date* columns are represented as strings, those needs to be converted as datetime.

Once cleansed, the descriptive statistics can be applied to the numerial columns, and they are Sales, Quantity, Discount and Profit.


function **text2float()** will take a txt number as a parameter and convert back to float64 number.

In [ ]:
def text2float(textnum, numwords={}):
    try:
        # Attempt to convert to float
        return float(textnum)
    except ValueError:
        # If conversion to float fails, continue with text to number conversion
        textnum = textnum.lower()
        
        if not numwords:
            units = [
                "zero", "one", "two", "three", "four", "five", "six", "seven", "eight",
                "nine", "ten", "eleven", "twelve", "thirteen", "fourteen", "fifteen",
                "sixteen", "seventeen", "eighteen", "nineteen",
            ]

            tens = ["", "", "twenty", "thirty", "forty", "fifty", "sixty", "seventy", "eighty", "ninety"]

            scales = ["hundred", "thousand", "million", "billion", "trillion"]

            numwords["and"] = (1, 0)
            for idx, word in enumerate(units):
                numwords[word] = (1, idx)
            for idx, word in enumerate(tens):
                numwords[word] = (1, idx * 10)
            for idx, word in enumerate(scales):
                numwords[word] = (10 ** (idx * 3 or 2), 0)

        current = result = 0
        for word in textnum.split():
            if word not in numwords:
                raise Exception("Illegal word: " + word)

            scale, increment = numwords[word]
            current = current * scale + increment
            if scale > 100:
                result += current
                current = 0

        return result + current

## 2. Task 2

Before performing any statistical analysis, the numerical column data has to be cleansed to be meaningful.
* Records with special characters on *Quantity* and needs to be cleansed. 
* Records with special characters on *Profit* and needs to be cleansed. 
* Applying the **text2float()** function to fix *Quantity* column. 

In [ ]:
# Removing "?" from Quantity column
global_super_store_data['Quantity'] = global_super_store_data['Quantity'].str.replace('?', '')

# Removing """ from Profit column
global_super_store_data['Profit'] = global_super_store_data['Profit'].str.replace('"', '')

# Assuming zero values for NaN on Profits
global_super_store_data['Profit'] = global_super_store_data['Profit'].fillna(0)


# Removing """ from Postal Code column
global_super_store_data['Postal Code'] = global_super_store_data['Postal Code'].str.replace('"', '')

# Applying text2float function
global_super_store_data['Quantity'] = global_super_store_data['Quantity'].apply(text2float)
global_super_store_data['Profit'] = global_super_store_data['Profit'].apply(text2float)
global_super_store_data

### 2.1 Descriptive Statistics

In [ ]:
# Row ID is not needed for the analysis, hence dropping the column
if 'Row ID' in global_super_store_data.columns:
    global_super_store_data.drop('Row ID', axis=1, inplace=True)

global_super_store_data.describe()

In [ ]:
# Columns with missing data
print(f"Sum of null records:\n{global_super_store_data.isnull().sum()}")

### 2.2 Outlier Treatment (Still needs to be worked on)

In [ ]:
mean_value = global_super_store_data['Sales'].mean()
std_value = global_super_store_data['Sales'].std()

# Define a threshold for identifying outliers (e.g., 3 standard deviations from the mean)
threshold = 3
lower_threshold = mean_value - threshold * std_value
upper_threshold = mean_value + threshold * std_value

print(f"mean: {mean_value}")
print(f"std: {std_value}")
print(f"lower_threshold: {lower_threshold}")
print(f"upper_threshold: {upper_threshold}")


# Filter outliers
outliers = global_super_store_data[(global_super_store_data['Sales'] < lower_threshold) | (global_super_store_data['Sales'] > upper_threshold)]
outliers

### 2.3 Normalizing and Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

#Select Numerical Features
numerical_features = ['Sales', 'Quantity', 'Discount', 'Profit']
global_super_store_numerial_data = global_super_store_data[numerical_features]

# # Min-Max Scaling
# scaler = MinMaxScaler()
# scaled_data = scaler.fit_transform(global_super_store_numerial_data)

# # Creating DataFrame with scaled data
# scaled_df = pd.DataFrame(scaled_data, columns=global_super_store_numerial_data.columns)

# print(f"Min-Max Scaled DataFrame:\n{scaled_df}")

# # Standard Scaling (Z-score normalization)
# scaler = StandardScaler()
# standardized_data = scaler.fit_transform(global_super_store_numerial_data)

# # Creating DataFrame with standardized data
# standardized_df = pd.DataFrame(standardized_data, columns=global_super_store_numerial_data.columns)

# print("\nStandardized DataFrame:")
# standardized_df


# Correlation matrix
correlation_matrix = global_super_store_numerial_data.corr()
correlation_matrix


### 2.4 Grouping of data

#### Grouping Sales/Porfits based on Categories

In [ ]:
unique_categories = global_super_store_data['Category'].unique();
unique_categories

In [33]:
# group total sales by category from the highest sale.
sales_category = global_super_store_data.groupby('Category')['Sales'].sum().sort_values(ascending=False)
sales_category

Category
Technology         836154.0330
Furniture          733206.6573
Office Supplies    718808.1040
Frnture              8621.0280
Name: Sales, dtype: float64

In [ ]:
# group total profits by category
profit_category = global_super_store_data.groupby('Category')['Profit'].sum().sort_values(ascending=False)
profit_category

In [ ]:
# group total sales by category
sales_category = global_super_store_data.groupby('Category')['Sales'].sum()

# group total profits by category
profit_category = global_super_store_data.groupby('Category')['Profit'].sum()


# figure size
plt.figure(figsize=(16,12));

# left total sales pie chart
plt.subplot(1,2,1); # 1 row, 2 columns, the 1st plot.
plt.pie(sales_category.values, labels=sales_category.index, startangle=90, counterclock=False,
        autopct=lambda p:f'{p:.1f}% \n £{p*np.sum(sales_category.values)/100 :,.0f}', 
        wedgeprops={'linewidth': 1, 'edgecolor':'black', 'alpha':0.75});
plt.axis('square');
plt.title('Total Sales by Category',  fontdict={'fontsize':16});

# right total profits pie chart
plt.subplot(1,2,2); # 1 row, 2 columns, the 2nd plot
plt.pie(profit_category.values, labels=profit_category.index, startangle=90, counterclock=False,
        autopct=lambda p:f'{p:.1f}% \n ${p*np.sum(profit_category.values)/100 :,.0f}',
        wedgeprops={'linewidth': 1, 'edgecolor':'black', 'alpha':0.75});
plt.axis('square');
plt.title('Total Profit by Category', fontdict={'fontsize':16});

### 2.5 Handling missing values in the dataset

### 2.6 Correlation 

### 2.7 Univariate analysis and visualisation

In [ ]:
columns_to_describe['Profit'] = columns_to_describe['Profit'].str.replace('"','')
columns_to_describe['Profit'] = columns_to_describe['Profit'].fillna(0)
columns_to_describe

In [ ]:
columns_to_describe.describe()
columns_to_describe.info()

In [ ]:
columns_to_describe['Profit'] = columns_to_describe['Profit'].apply(text2float)
columns_to_describe

In [ ]:
columns_to_describe.describe()

In [ ]:
columns_to_describe.info()

In [ ]:
columns_to_describe['Transformed'] = np.log1p(columns_to_describe['Sales'])
print("Original data:")
columns_to_describe

In [ ]:
ss_data.describe()

In [ ]:
ss_data['Profit'] = ss_data['Profit'].str.replace('"', '')

In [ ]:
ss_data['Quantity'] = ss_data['Quantity'].str.replace('?', '')

In [ ]:
ss_data['Quantity'] = ss_data['Quantity'].apply(text2float)

In [ ]:
ss_data.describe()

In [ ]:
ss_data['Postal Code'] = ss_data['Postal Code'].str.replace('"', '')

In [ ]:
ss_data['Country'] = ss_data['Country'].str.replace('US', 'United States')

In [ ]:
ss_data.to_csv('10042024.csv')

In [ ]:
ss_data['State'] = ss_data['State'].dropna()
us_states = ss_data['State'].unique()

print(us_states)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# %matplotlib inline

In [ ]:
sns.pairplot(ss_data)

In [ ]:
ss_data['Category'] = ss_data['Category'].str.replace('Frnture', 'Furniture')

In [ ]:
sns.pairplot(ss_data)

In [ ]:
ss_data.groupby('State').size()

In [ ]:
sns.lmplot(data=ss_data, x='Sales', y='Quantity', hue='Category')